In [1]:
import tushare as ts
print(f"tushare version:{ts.__version__}")
token = "8ef5ec61cdd848715c57c11d58dd71da1271f76b2420d2bac8aef123"
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False
import pandas as pd
import numpy  as np
import random

tushare version:1.2.54


In [2]:
class TushareData(object):
    def __init__(self, exchange, token):
        self.exchange = exchange
        self.pro = ts.pro_api(token)
        self.ml_ts_codes = None
        self.all_ts_codes =None
    def getData(self, start_date = None, end_date =None):
        self._getSymbols()
        self._getOHLC(start_date, end_date)
        self._getTFMR(start_date, end_date)
        if self.TFMR_all.empty:
            ALL = self.Symbols_all.merge(self.OHLC_all, on ='ts_code')
        else:
            ALL = self.Symbols_all.merge(self.OHLC_all, on ='ts_code').merge(self.TFMR_all, on = 'ts_code')
        if self.TFMR_ml.empty:
            ML = self.Symbols_ml.merge(self.OHLC_ml, on = 'ts_code')
        else:
            ML = self.Symbols_ml.merge(self.OHLC_ml, on = 'ts_code').merge(self.TFMR_ml, on = 'ts_code')
        return ML, ALL
        
    def _getSymbols(self):
        """
        Process the name and symbols
        """
        mc_ml = self.pro.fut_basic(exchange=self.exchange, fut_type='2', fields='ts_code,symbol,name,list_date,delist_date')
        mc_ml.name = mc_ml.name.str[:-4]
        

        mc_ord = self.pro.fut_basic(exchange=self.exchange, fut_type = '1', fields='ts_code,symbol,name,list_date,delist_date')
        mc_ord.name = mc_ord.name.str[:-4]
        
        mcs = [mc_ml, mc_ord]
        for mc in mcs:
            mc.loc[mc.loc[mc.name =='螺纹'].index, 'name'] = '螺纹钢'
            mc.loc[mc.loc[mc.name =='燃油'].index, 'name'] = '燃料油'
            if self.exchange.upper() == 'SHFE':
                mc.loc[mc.loc[mc.name =='沪金'].index, 'name'] = '黄金'
                mc.loc[mc.loc[mc.name =='沪镍'].index, 'name'] = '镍'
                mc.loc[mc.loc[mc.name =='沪锡'].index, 'name'] = '锡'
                mc.loc[mc.loc[mc.name =='沪铜'].index, 'name'] = '铜'
                mc.loc[mc.loc[mc.name =='沪铜'].index, 'name'] = '铜'
                mc.loc[mc.loc[mc.name =='沪银'].index, 'name'] = '银'
                mc.loc[mc.loc[mc.name =='沪铝'].index, 'name'] = '铝'
                mc.loc[mc.loc[mc.name =='沪锌'].index, 'name'] = '锌'
                mc.loc[mc.loc[mc.name =='沪铅'].index, 'name'] = '铅'
            elif self.exchange.upper() == 'DCE':
                pass
            elif self.exchange.upper() == 'INE':
                pass
            elif self.exchange.upper() == 'CFFEX':
                pass
            elif self.exchange.upper() == 'CZCE':
                pass
        ## 主力连续
        self.Symbols_ml = mc_ml.iloc[:,:-2]
        self.ml_ts_codes = list(self.Symbols_ml.ts_code.unique())
        ## 普通合约
        self.Symbols_ord = mc.iloc[:,:-2]
        self.ord_ts_codes = list(self.Symbols_ord.ts_code.unique())
        ## 全部合约
        self.all_ts_codes = self.ord_ts_codes + self.ml_ts_codes
        self.Symbols_all = pd.concat([self.Symbols_ord, self.Symbols_ml])
            
    def _getOHLC(self, start_date = None, end_date =None):
        """
        get Open, High, Low, Close
        """
        if not start_date or not end_date:
            print("Missed time data, format:'yyyymmdd'")
            return
        cols = self.pro.fut_daily(ts_code = "CU.SHF", start_date ="20180101", end_date ="20180101" ).columns
        
        ## 主力连续合约
        ml_OHLC = pd.DataFrame(columns = cols)
        for ts_c in self.ml_ts_codes:
            daily = self.pro.fut_daily(ts_code = ts_c, start_date = start_date, end_date = end_date)
            ml_OHLC = pd.concat([ml_OHLC, daily])
            if len(daily):
                print(f"Successfully get {ts_c} data! -- {len(daily)} rows", end = " / ")
                continue
            print("XXXX", end = " / ")

        ml_OHLC.reset_index(drop = True, inplace = True)
        self.OHLC_ml = ml_OHLC.drop(columns = ['pre_close','pre_settle',"change1","change2","oi","oi_chg"])
        self.OHLC_ml.amount *= 10000
        self.OHLC_ml.dropna().reset_index(drop = True)
        
        ## 全部合约
        all_OHLC = pd.DataFrame(columns = cols)
        for ts_c in self.all_ts_codes[-100]:
            daily = self.pro.fut_daily(ts_code = ts_c, start_date = start_date, end_date = end_date)
            all_OHLC = pd.concat([all_OHLC, daily])
            if len(daily):
                print(f"Successfully get {ts_c} data! -- {len(daily)} rows", end = " / ")
                continue
            print("XXXX", end = " / ")

        all_OHLC.reset_index(drop = True, inplace = True)
        self.OHLC_all = all_OHLC.drop(columns = ['pre_close','pre_settle',"change1","change2","oi","oi_chg"])
        self.OHLC_all.amount *= 10000
        self.OHLC_all.dropna().reset_index(drop = True)
        return
    
    def _getTFMR(self,start_date = None, end_date = None):
        """
        get Trade Fees and Margin Rate
        """
        all_settle = self.pro.fut_settle(start_date = start_date, end_date = end_date, exchange = self.exchange)
        drop_ind_ml = []
        drop_ind_all =[]
        ### 记录主力连续与全部合约的有效数据
        for index, row in all_settle.iterrows():
            if row.ts_code in self.ord_ts_codes:
                drop_ind_ml.append(index)
            elif row.ts_code not in self.all_ts_codes:
                drop_ind_all.append(index)
        print(f"Among all {len(all_settle)} data, {len(drop_ind_ml)} will be removed for '主力连续合约' and {len(drop_ind_all)} will be removed for '全部合约'")
        
        ## 主力连续合约
        ml = all_settle.drop(drop_ind_ml) \
                              .reset_index(drop = True) \
                              .drop(columns = ['trading_fee_rate', 'delivery_fee'])
        ml['margin_rate'] = 100*0.25*(ml.b_hedging_margin_rate 
                                    + ml.s_hedging_margin_rate 
                                    + ml.long_margin_rate 
                                    + ml.short_margin_rate)

        ml = ml.drop(columns = ['b_hedging_margin_rate', 's_hedging_margin_rate', 'long_margin_rate', 'short_margin_rate'])
        self.TFMR_ml = ml
        ## 全部合约
        ml = all_settle.drop(drop_ind_all) \
                              .reset_index(drop = True) \
                              .drop(columns = ['trading_fee_rate', 'delivery_fee'])
        ml['margin_rate'] = 100*0.25*(ml.b_hedging_margin_rate 
                                    + ml.s_hedging_margin_rate 
                                    + ml.long_margin_rate 
                                    + ml.short_margin_rate)

        ml = ml.drop(columns = ['b_hedging_margin_rate', 's_hedging_margin_rate', 'long_margin_rate', 'short_margin_rate'])
        self.TFMR_all = ml
    

In [3]:
t = TushareData(exchange = 'SHFE', token = token)
SHFE_ml, SHFE_all = t.getData("20180101", "20181231")

XXXX / Successfully get AUL.SHF data! -- 243 rows / Successfully get AL.SHF data! -- 243 rows / Successfully get SNL.SHF data! -- 243 rows / Successfully get CU.SHF data! -- 243 rows / Successfully get ZN.SHF data! -- 243 rows / Successfully get AG.SHF data! -- 243 rows / Successfully get RB.SHF data! -- 243 rows / Successfully get BUL.SHF data! -- 243 rows / Successfully get SN.SHF data! -- 243 rows / Successfully get FUL.SHF data! -- 230 rows / Successfully get AGL.SHF data! -- 243 rows / Successfully get NI.SHF data! -- 243 rows / Successfully get HCL.SHF data! -- 243 rows / Successfully get PBL.SHF data! -- 243 rows / Successfully get WR.SHF data! -- 218 rows / Successfully get NIL.SHF data! -- 243 rows / Successfully get RBL.SHF data! -- 243 rows / Successfully get CUL.SHF data! -- 243 rows / Successfully get FU.SHF data! -- 230 rows / Successfully get AU.SHF data! -- 243 rows / Successfully get ALL.SHF data! -- 243 rows / Successfully get PB.SHF data! -- 243 rows / Successfully g

Exception: trade_date,ts_code不能都为空

In [5]:
SHFE_ml

NameError: name 'SHFE_ml' is not defined

In [15]:
SHFE_all

,symbol,name,trade_date_x,open,high,low,close,settle_x,vol,amount,ts_code,trade_date_y,settle_y,trading_fee,margin_rate
